In [137]:
import os
import audioUtils
import keras
import librosa
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.utils import to_categorical
import numpy as np
import tqdm

In [175]:
DATA_PATH = "../../DeadSimpleSpeechRecognizor_Data/"
DYData_Path = "../DysSpeech_Corpora/Linzy/linzyCut/train/"
DYSCmdCategs = {'one': 0, 'two' : 1, 'three' : 2, 'four' : 3, 'five' : 4, 'six' : 5, 'seven' : 6, 'eight' : 7, 'nine' : 8, 'close' : 9, 'up' : 10,
                    'down' : 11, 'previous' : 12, 'next' : 13, 'in' : 14, 'out' : 15, 'left' : 16, 'right' : 17, 'home' : 18}
#DYSCmdCategNum = [value for value in DYSCmdCategs.values()]

inverseDYSCmdCategs = {0: 'one', 1: 'two', 2: 'three', 3: 'four', 4: 'five', 5: 'six', 6: 'seven', 7: 'eight', 8: 'nine', 9: 'close', 10: 'up', 11: 'down', 12: 'previous', 13: 'next', 14: 'in', 15: 'out', 16: 'left', 17: 'right', 18: 'home'}
#{v: k for k, v in DYSCmdCategs.items()}
#print(inverseDYSCmdCategs)

In [176]:
DYSCmdCategString = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'close', 'up', 'down', 'previous', 'next', 'in', 'out', 'left', 'right', 'home']#[lbl for lbl in DYSCmdCategs]
DYSCmdCategNum = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
print(DYSCmdCategNum)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [177]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

In [181]:
# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DYData_Path):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)

In [179]:
feature_dim_1 = 20
feature_dim_2 = 11
channel = 1
epochs = 100
batch_size = 100
verbose = 1
num_classes = 19 #categories

In [180]:
def run_main():
    lbls, idices, encodingData = get_labels(path = DATA_PATH)
    print("The labels is :",lbls)
    print("the label indices is:", idices)
    print("the encoded data is:", encodingData)
    print("Encoding the Dyspeech commands labels.......")
    dyEncodedData = to_categorical(DYSCmdCategNum)
    print("the encoded speech command labels are:", dyEncodedData)
    print("Encoded Labels' shape is:", dyEncodedData.shape)
    

In [184]:
def run_main2():
    print(get_labels())

In [185]:
if __name__ == "__main__":
    run_main2()

(['14next', '.DS_Store', '18right', '11up', 'extract_loudest_section', '9', '7', '17left', '6', '1', '8', '15in', '10close', '16out', '19home', '4', '3', '13previous', '2', '5', '12down'], array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20]), array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
    

# 以下測試MFCC

In [145]:
basePathOfTrainData = "../DysSpeech_Corpora/Linzy/linzyCut/train/"

In [146]:
def getNextLevelDirs(root):
     return next(os.walk(root))[1]

def getAllWavFiles():
    _baseDir = '../DysSpeech_Corpora/Linzy/linzyCut/train/'
    firstLevelDirs = getNextLevelDirs(_baseDir)
    trainfiles = list()
    for folder in firstLevelDirs:
        d = os.path.join(_baseDir,folder)
        #print(d)
        #files = [os.path.join(d,f+'.npy') for f in next(os.walk(d))[2] if f.endswith('.wav')]
        files = [os.path.join(d,f) for f in next(os.walk(d))[2] if f.endswith('.wav')]
        #print(files)
        #trainfiles.append([os.path.join(d,f+'.npy') for f in next(os.walk(d))[2] if f.endswith('.wav')])
        trainfiles += files
    
def getAllFolder():
    _baseDir = '../DysSpeech_Corpora/Linzy/linzyCut/train/'
    firstLevelDirs = getNextLevelDirs(_baseDir)
    print("firstLevelDirs : ", firstLevelDirs)
    return firstLevelDirs

ALL_Folders = getAllFolder()   

firstLevelDirs :  ['14next', '18right', '11up', '9', '7', '17left', '6', '1', '8', '15in', '10close', '16out', '19home', '4', '3', '13previous', '2', '5', '12down']


In [213]:
#labelDict = {1:'one','14next':, '18right', '11up', '17left', , '15in', '10close', '16out', '19home',, '13previous', '12down'}
folderList = ['1','2','3','4','5','6','7','8','9','10', '11', '12','13','14', '15','16','17','18','19']
DSCmdLabels = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'close', 'up', 'down', 'previous', 'next', 'in', 'out', 'left', 'right', 'home']
def save_data_to_array(path=basePathOfTrainData, max_len=11):
    #labels, _, _ = get_labels(path)

    for folder in folderList:#DYSCmdCategString:
        # Init mfcc vectors
        mfcc_vectors = []
        wavfiles = [basePathOfTrainData + folder +"/"+ wavfile for wavfile in os.listdir(path + folder+'/') if wavfile.endswith('.wav')]#if wavfile.endswith('.wav')
        #wavfiles = [wavfile for wavfile in ]
        print("Current folder is : ",folder)
        #print("the whole files are : ",wavfiles)
        #print("+++++++++++++++++++++++++++++++++++++")
        for wavfile in wavfiles:#tqdm(wavfiles, "Saving vectors of label - '{}'".format(folder)):
            mfcc_ = wav2mfcc(wavfile)
            mfcc_vectors.append(mfcc_)
        print("length of mfcc_vectors is : ",len(mfcc_))
        np.save(folder + '.npy', mfcc_vectors)

In [214]:
save_data_to_array()

Current folder is :  1
length of mfcc_vectors is :  20
Current folder is :  2
length of mfcc_vectors is :  20
Current folder is :  3
length of mfcc_vectors is :  20
Current folder is :  4
length of mfcc_vectors is :  20
Current folder is :  5
length of mfcc_vectors is :  20
Current folder is :  6
length of mfcc_vectors is :  20
Current folder is :  7
length of mfcc_vectors is :  20
Current folder is :  8
length of mfcc_vectors is :  20
Current folder is :  9
length of mfcc_vectors is :  20
Current folder is :  10
length of mfcc_vectors is :  20
Current folder is :  11
length of mfcc_vectors is :  20
Current folder is :  12
length of mfcc_vectors is :  20
Current folder is :  13
length of mfcc_vectors is :  20
Current folder is :  14
length of mfcc_vectors is :  20
Current folder is :  15
length of mfcc_vectors is :  20
Current folder is :  16
length of mfcc_vectors is :  20
Current folder is :  17
length of mfcc_vectors is :  20
Current folder is :  18
length of mfcc_vectors is :  20
C

In [87]:
#read all the audio data and convert them into mfcc
testFile = DYData_Path+"13previous/LinZY03_13_3.wav"
def testWavToMFCC(anAudioFile):
    mfcc = wav2mfcc(anAudioFile)
    return mfcc

#_mfcc = testWavToMFCC(testFile)
    
#print("type of mfcc : ",type(_mfcc))
#print("shape of mfcc : ",_mfcc.shape)

In [209]:
from subprocess import call
def extract_loudest_section(path=basePathOfTrainData):
    #labels, _, _ = get_labels(path)
    targetD = 0
    for folder in folderList:#DYSCmdCategString:
        # Init mfcc vectors
        #mfcc_vectors = []
        targetD +=1
        targetDName = "./template/dir/" + str(targetD)
        wavfiles = [basePathOfTrainData + folder +"/"+ wavfile for wavfile in os.listdir(path + folder+'/') if wavfile.endswith('.wav')]#if wavfile.endswith('.wav')
        #wavfiles = [wavfile for wavfile in ]
        print("Current folder is : ",folder)
        #print("the whole files are : ",wavfiles)
        #print("+++++++++++++++++++++++++++++++++++++")
        
        for wavfile in wavfiles:#tqdm(wavfiles, "Saving vectors of label - '{}'".format(folder)):
            #mfcc_ = wav2mfcc(wavfile)
            #mfcc_vectors.append(mfcc_)
            #wavfile = os.path.splitext(wavfile)[0]
            print("processing file {}".format(wavfile))
            call(["./extract_loudest_section", wavfile, targetDName])
            
        #print("length of mfcc_vectors is : ",len(mfcc_))
        #np.save(folder + '.npy', mfcc_vectors)

In [210]:
extract_loudest_section()

Current folder is :  1
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/1/LinZY03_01_1.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/1/LinZY03_01_2.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/1/LinZY03_01_3.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/1/LinZY03_01_4.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/1/LinZY03_01_5.wav
Current folder is :  2
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/2/LinZY03_02_1.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/2/LinZY03_02_3.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/2/LinZY03_02_2.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/2/LinZY03_02_5.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/2/LinZY03_02_4.wav
Current folder is :  3
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/3/LinZY03_03_3.wav
processing file ../DysSpeech_Corpora/Linzy/linzyCut/train/3/LinZY03_03_2.wav
process

In [201]:
def makedir():
    basepath = "./template/dir/"
    for idx in range(19):
        dirname = basepath+str((idx+1))
        print("making dir {}".format(dirname))
        os.makedirs(dirname)

In [202]:
makedir()

making dir ./template/dir/1
making dir ./template/dir/2
making dir ./template/dir/3
making dir ./template/dir/4
making dir ./template/dir/5
making dir ./template/dir/6
making dir ./template/dir/7
making dir ./template/dir/8
making dir ./template/dir/9
making dir ./template/dir/10
making dir ./template/dir/11
making dir ./template/dir/12
making dir ./template/dir/13
making dir ./template/dir/14
making dir ./template/dir/15
making dir ./template/dir/16
making dir ./template/dir/17
making dir ./template/dir/18
making dir ./template/dir/19


In [215]:
wavfiles2 = ["../DysSpeech_Corpora/Linzy/linzyCut/test/"+ wavfile for wavfile in os.listdir("../DysSpeech_Corpora/Linzy/linzyCut/test/") 
             if wavfile.endswith('.wav')]
for wavfile2 in wavfiles2:
    call(["./extract_loudest_section", wavfile2, "../DysSpeech_Corpora/Linzy/linzyCut/test2/"])
    